<a href="https://colab.research.google.com/github/krackalackel02/CS-M148/blob/main/L04_DSLC_prediction_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting sale price in Ames, IA using LAD and LS

## [DSLC stages]: Analysis


The following code sets up the libraries and creates cleaned and pre-processed training, validation and test data that we will use in this document.

In [ ]:

%pip install scikit-lego

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.8/180.8 kB 7.1 MB/s eta 0:00:00


In [ ]:
#Add code to look at the TSV file...

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklego.linear_model import LADRegression



pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 100

In [ ]:
ames_train_processed = pd.read_csv('ames_train_preprocessed.csv', index_col=0)


In [ ]:
#add code to load the test and validation data sets

In [ ]:
ames_train_processed

,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod_add,mas_vnr_area,exter_qual,bsmt_qual,bsmt_exposure,total_bsmt_sf,heating_qc,gr_liv_area,bedroom_abvgr,kitchen_qual,totrms_abvgrd,fireplaces,fireplace_qu,garage_yr_blt,garage_finish,garage_cars,garage_area,wood_deck_sf,mo_sold,yr_sold,saleprice,masonry_veneer_brick,foundation_cinder,foundation_concrete,garage_attached,lot_inside,exterior_vinyl,bathrooms,porch,residential_density,irregular_lot_shape,house_floors,basement_finished_rating,neighborhood_BrkSide,neighborhood_CollgCr,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NWAmes,neighborhood_OldTown,neighborhood_Sawyer,neighborhood_Somerst
pid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
526351030,87.0,11029,6,8,1958,2002,0.0,5,4,1,1184,5,1414,3,3,6,1,3,2002,1,2,601,0,5,2008,176500,0,1,0,1,0,0,2.0,1,1,1,1.0,5,False,False,False,False,False,True,False,False,False,False
526353050,68.0,12925,6,7,1970,1970,0.0,3,3,2,1205,5,2117,4,3,7,2,4,1970,3,2,550,0,5,2008,237500,0,1,0,1,0,0,2.5,1,1,1,1.0,3,False,False,False,False,False,True,False,False,False,False
526354070,85.0,11075,6,5,1969,1969,0.0,3,2,2,952,3,2112,4,3,9,2,4,1969,1,2,576,280,6,2008,206900,0,1,0,1,1,0,2.5,0,1,0,2.0,5,False,False,False,False,False,True,False,False,False,False
527105050,72.0,8702,6,5,1997,1998,0.0,3,3,1,926,5,1604,3,3,7,1,3,1998,3,2,470,0,4,2008,187500,0,0,1,1,1,1,2.5,1,1,1,2.0,3,False,False,False,True,False,False,False,False,False,False
527106140,59.0,9535,6,5,1998,1998,0.0,3,4,4,926,5,1604,3,3,7,0,0,1998,3,2,472,100,5,2008,195500,0,0,1,1,1,1,2.5,1,1,1,2.0,6,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923250060,80.0,17400,5,5,1977,1977,0.0,3,3,1,1126,2,1126,3,3,5,1,4,1977,2,2,484,295,5,2006,160000,0,1,0,1,1,0,3.0,1,1,0,1.0,5,False,False,False,False,True,False,False,False,False,False
923275080,37.0,7937,6,6,1984,1984,0.0,3,3,3,1003,3,1003,3,3,6,0,0,1984,1,2,588,120,3,2006,142500,0,1,0,0,0,0,2.0,0,1,1,1.0,6,False,False,False,False,True,False,False,False,False,False
923276100,68.0,8885,5,5,1983,1983,0.0,3,4,3,864,3,902,2,3,5,0,0,1983,1,2,484,164,6,2006,131000,0,1,0,1,1,0,2.0,0,1,1,1.0,3,False,False,False,False,True,False,False,False,False,False



## Single predictor fits

### LAD fit to the sample of 10 houses

Let's first demonstrate generating a LAD fit for the sample of 10 houses.

First let's create the sample of 10 houses that we used in the book.

In [ ]:
sample_pid = [534479240, 535478110, 905106150, 905352030, 909250120, 907280090, 916386040, 528235160, 535176050, 908128050]
sample_ames_clean = ames_train_processed.loc[sample_pid]
sample_ames_clean

,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod_add,mas_vnr_area,exter_qual,bsmt_qual,bsmt_exposure,total_bsmt_sf,heating_qc,gr_liv_area,bedroom_abvgr,kitchen_qual,totrms_abvgrd,fireplaces,fireplace_qu,garage_yr_blt,garage_finish,garage_cars,garage_area,wood_deck_sf,mo_sold,yr_sold,saleprice,masonry_veneer_brick,foundation_cinder,foundation_concrete,garage_attached,lot_inside,exterior_vinyl,bathrooms,porch,residential_density,irregular_lot_shape,house_floors,basement_finished_rating,neighborhood_BrkSide,neighborhood_CollgCr,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NWAmes,neighborhood_OldTown,neighborhood_Sawyer,neighborhood_Somerst
pid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
534479240,79.0,9462,5,6,1949,1973,0.0,3,3,1,704,4,1728,3,3,7,1,4,1973,1,1,234,245,7,2006,137000,0,1,0,1,1,0,1.5,1,1,0,2.0,1,False,False,False,False,False,True,False,False,False,False
535478110,70.0,7000,5,5,1962,1962,0.0,3,3,3,1025,3,1025,3,3,6,0,0,1962,0,0,0,96,3,2006,124000,0,1,0,0,1,0,2.0,1,1,0,1.0,5,False,False,False,False,False,True,False,False,False,False
905106150,109.0,8724,5,5,1968,1968,0.0,3,4,1,894,4,894,3,3,5,1,1,1968,3,2,450,0,5,2007,129000,0,1,0,1,1,1,1.0,0,1,0,1.0,3,False,False,False,False,False,False,False,False,True,False
905352030,83.0,10005,7,5,1977,1977,299.0,3,4,1,1160,5,2022,4,3,8,1,3,1977,3,2,505,288,3,2008,192000,1,1,0,1,1,0,2.5,1,1,0,2.0,3,False,False,False,False,False,False,False,False,False,False
909250120,96.0,13132,5,5,1914,1950,0.0,3,4,2,747,4,1784,4,3,9,0,0,1914,1,1,180,203,7,2006,138887,0,0,0,0,1,0,1.5,1,1,0,2.0,1,False,False,False,False,False,False,False,False,False,False
907280090,68.0,15295,7,5,1996,2002,254.0,4,4,2,860,5,1992,3,4,7,2,3,2002,2,2,608,225,6,2007,211000,1,0,1,1,0,1,3.5,1,1,3,2.0,6,False,True,False,False,False,False,False,False,False,False
916386040,73.0,9802,5,5,2006,2007,0.0,3,4,1,352,4,1442,3,3,6,0,0,2007,3,2,400,100,4,2008,172500,0,0,1,1,1,1,2.5,0,1,0,1.0,1,False,False,False,False,False,False,False,False,False,False
528235160,59.0,11796,7,5,2004,2005,0.0,4,4,3,847,5,1959,4,4,8,1,4,2005,3,2,434,100,7,2007,215000,0,0,1,1,1,1,2.5,1,1,1,2.0,1,False,False,False,True,False,False,False,False,False,False
535176050,80.0,11600,6,5,1960,1960,175.0,3,3,1,1383,3,1383,3,3,7,0,0,1960,2,1,292,0,4,2006,145250,1,1,0,1,1,0,1.5,1,1,0,1.0,4,False,False,False,False,False,True,False,False,False,False


In [ ]:
# create a clean version of the preview dataset and select only the saleprice and gr_live_area columns
sample_ames_clean = sample_ames_clean[['saleprice', 'gr_liv_area']]
sample_ames_clean

,saleprice,gr_liv_area
pid,,
534479240,137000,1728
535478110,124000,1025
905106150,129000,894
905352030,192000,2022
909250120,138887,1784
907280090,211000,1992
916386040,172500,1442
528235160,215000,1959
535176050,145250,1383


Let's plot the living area against sale price using a scatterplot.

In [ ]:
px.scatter(sample_ames_clean, x='gr_liv_area', y='saleprice')


Next, we can fit a LAD line as follows:

In [ ]:
lad_fit = LADRegression()
lad_fit.fit(X=np.array(sample_ames_clean['gr_liv_area']).reshape(-1, 1),
            y=sample_ames_clean['saleprice'])
lad_fit.intercept_, lad_fit.coef_[0]

(60972.07200291597, 64.79918143401098)

Note that because we have only one predictor variable we have to reshape the `gr_liv_area` column into a `fit()` compatible array.


We can plot our LAD fitted line on our scatterplot:

In [ ]:
# plot the LAD fitted line on top of the scatterplot
fig = px.scatter(sample_ames_clean, x='gr_liv_area', y='saleprice')
fig.add_trace(
    go.Scatter(x=sample_ames_clean['gr_liv_area'],
               y=lad_fit.intercept_ + sample_ames_clean['gr_liv_area'] * lad_fit.coef_[0],
               mode='lines')
)


And we can predict the price of a $2000 ft^2$ house as follows (note that we rounded each coefficient to the nearest integer in the book, so the prediction is slightly different):


In [ ]:
# predict the sale price of a house with 1,000 square feet of living area using the LAD model
lad_fit.predict(np.array(2000).reshape(-1, 1))

array([190570.43487094])


### LS fit to the sample of 10 houses

To apply least squares to the 10 training data houses, we use the `linearRegression()` function from scikit-learn.

We can fit a LS line as follows:

In [ ]:
ls_fit = LinearRegression()
ls_fit.fit(X=np.array(sample_ames_clean['gr_liv_area']).reshape(-1, 1),
                y=sample_ames_clean['saleprice'])
ls_fit.intercept_, ls_fit.coef_[0]

(61190.177180895145, 64.2592620500356)

We can similarly plot our LS fitted line on our scatterplot on top of the LAD line (the two lines are very similar):

In [ ]:
# plot the LAD and LS fitted line on top of the scatterplot
fig = px.scatter(sample_ames_clean, x='gr_liv_area', y='saleprice')
fig.add_trace(
    go.Scatter(x=sample_ames_clean['gr_liv_area'],
               y=lad_fit.intercept_ + sample_ames_clean['gr_liv_area'] * lad_fit.coef_[0],
               mode='lines',
               name='LAD',
               line={'dash': 'dash',
                     'color': 'black'})
)
fig.add_trace(
    go.Scatter(x=sample_ames_clean['gr_liv_area'],
               y=ls_fit.intercept_ + sample_ames_clean['gr_liv_area'] * ls_fit.coef_[0],
               mode='lines',
               name='LS',
               line={'dash': 'solid',
                     'color': 'black'})
)


And we can predict the price of a $2000 ft^2$ house as follows (note that we rounded each coefficient to the nearest integer in the book, so the prediction is slightly different):

In [ ]:
ls_fit.predict(np.array(2000).reshape(-1, 1))

array([189708.70128097])


### LS and LAD fit to the full training set (single predictor)

Let's now fit LS and LAD to the entire training dataset using sale price as the sole predictor variable.

In [ ]:
lad_area_fit = LADRegression()
lad_area_fit.fit(X=np.array(ames_train_processed['gr_liv_area']).reshape(-1, 1),
                            y=ames_train_processed['saleprice'])

LADRegression()

In [ ]:
ls_area_fit = LinearRegression()
ls_area_fit.fit(X=np.array(ames_train_processed['gr_liv_area']).reshape(-1, 1),
                          y=ames_train_processed['saleprice'])

LinearRegression()


And we can plot both lines on a scatterplot:

In [ ]:
# plot both the LS and LAD lines on top of the scatterplot of gr_liv_area against sale_price
fig = px.scatter(ames_train_processed, x='gr_liv_area', y='saleprice')
fig.add_trace(
    go.Scatter(x=ames_train_processed['gr_liv_area'],
                y=lad_area_fit.intercept_ + ames_train_processed['gr_liv_area'] * lad_area_fit.coef_[0],
                mode='lines',
                name='LAD',
                line={'dash': 'dash',
                      'color': 'black'})
)
fig.add_trace(
    go.Scatter(x=ames_train_processed['gr_liv_area'],
                y=ls_area_fit.intercept_ + ames_train_processed['gr_liv_area'] * ls_area_fit.coef_[0],
                mode='lines',
                name='LS',
                line={'dash': 'solid',
                      'color': 'black'})
)



### Training performance

Below we create a table containing the observed and predicted sale price for each training set house.


In [ ]:
pred_train_df = pd.DataFrame(
    {'true': ames_train_processed['saleprice'],
     'ls_pred': ls_area_fit.predict(np.array(ames_train_processed['gr_liv_area']).reshape(-1, 1)),
     'lad_pred': lad_area_fit.predict(np.array(ames_train_processed['gr_liv_area']).reshape(-1, 1))})
pred_train_df

,true,ls_pred,lad_pred
pid,,,
526351030,176500,168068.625598,166877.618523
526353050,237500,244961.674115,235084.895259
526354070,206900,244414.781878,234599.779493
527105050,187500,188850.530603,185312.017641
527106140,195500,188850.530603,185312.017641
...,...,...,...
923250060,160000,136567.632749,138934.950386
923275080,142500,123114.083720,127001.102536
923276100,131000,112066.860534,117201.764057


Let's compute the rMSE, MAE, MAD, correlation and $R^2$ evaluations for each algorithm.

In [ ]:
# calculate the rMSE, MAE, MAD, correlation, and R2 of the true price with the LS and LAD predictions
print('LS rMSE:', np.sqrt(mean_squared_error(pred_train_df['true'], pred_train_df['ls_pred'])))
print('LS MAE:', mean_absolute_error(pred_train_df['true'], pred_train_df['ls_pred']))
print('LS MAD:', np.median(np.abs(pred_train_df['true'] - pred_train_df['ls_pred'])))
print('LS correlation:', np.corrcoef(pred_train_df['true'], pred_train_df['ls_pred'])[0, 1])
print('LS R2:', r2_score(pred_train_df['true'], pred_train_df['ls_pred']))

print('LAD rMSE:', np.sqrt(mean_squared_error(pred_train_df['true'], pred_train_df['lad_pred'])))
print('LAD MAE:', mean_absolute_error(pred_train_df['true'], pred_train_df['lad_pred']))
print('LAD MAD:', np.median(np.abs(pred_train_df['true'] - pred_train_df['lad_pred'])))
print('LAD correlation:', np.corrcoef(pred_train_df['true'], pred_train_df['lad_pred'])[0, 1])
print('LAD R2:', r2_score(pred_train_df['true'], pred_train_df['lad_pred']))


LS rMSE: 44492.147975913846
LS MAE: 31077.300371087003
LS MAD: 22420.37538449798
LS correlation: 0.7746966958178205
LS R2: 0.6001549705110487
LAD rMSE: 44951.997754399
LAD MAE: 30658.05586018359
LAD MAD: 21531.69790517607
LAD correlation: 0.7746966958178201
LAD R2: 0.5918470399525836



Across all metrics, the two algorithms are fairly similar, but, unsurprisingly, the LS algorithm looks better when evaluating using the rMSE, whereas the LAD algorithm looks better when evaluating using the MAE and MAD. This is unsurprising because the LS algorithm is designed to minimize the squared loss (which the rMSE computes), while the LAD algorithm is designed to minimize the absolute value loss (which the MAE and MAD compute).

Below, we also plot the predicted versus observed sale price responses for the training data, and add a diagonal line corresponding to the perfect prediction. The two algorithms yield very similar predictions to one another, and the predictions themselves are fairly similar to the observed responses.

In [ ]:
# create the plot below for LAD instead
lad_fig = px.scatter(pred_train_df, x='true', y='lad_pred')
# add a line with slope 1 and intercept 0
lad_fig.add_trace(
    go.Scatter(x=[0, 600000], y=[0, 600000], mode='lines'))
# update axes names and ranges
lad_fig.update_layout(xaxis_title='Observed sale price',
                        yaxis_title='Predicted sale price',
                        title="LAD pred",
                        xaxis_range=[0, 800000],
                        yaxis_range=[0, 800000])


In [ ]:
ls_fig = px.scatter(pred_train_df, x='true', y='ls_pred')
# add a line with slope 1 and intercept 0
ls_fig.add_trace(
    go.Scatter(x=[0, 600000], y=[0, 600000], mode='lines'))
# update axes names and ranges
ls_fig.update_layout(xaxis_title='Observed sale price',
                        yaxis_title='Predicted sale price',
                        title="LS pred",
                        xaxis_range=[0, 800000],
                        yaxis_range=[0, 800000])



### PCS evaluation

#### Predictability (validation performance)

While the training performance is informative, if we want to get a sense of how well the algorithm would perform on future data we actually need to evaluate the algorithms on the validation set (which is chosen to be as similar as possible to the future data). Recall that we used a time-split to separate the dataset into training, validation and test sets.

Let's repeat the analysis above, but this time for the validation set houses.


First we need to generate predictions for the validation set houses:

In [ ]:
ames_val_preprocessed = pd.read_csv('ames_val_preprocessed.csv', index_col=0)

In [ ]:
ames_val_preprocessed

,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod_add,mas_vnr_area,exter_qual,bsmt_qual,bsmt_exposure,total_bsmt_sf,heating_qc,gr_liv_area,bedroom_abvgr,kitchen_qual,totrms_abvgrd,fireplaces,fireplace_qu,garage_yr_blt,garage_finish,garage_cars,garage_area,wood_deck_sf,mo_sold,yr_sold,saleprice,masonry_veneer_brick,foundation_cinder,foundation_concrete,garage_attached,lot_inside,exterior_vinyl,bathrooms,porch,residential_density,irregular_lot_shape,house_floors,basement_finished_rating,neighborhood_BrkSide,neighborhood_CollgCr,neighborhood_Edwards,neighborhood_Gilbert,neighborhood_Mitchel,neighborhood_NAmes,neighborhood_NWAmes,neighborhood_OldTown,neighborhood_Sawyer,neighborhood_Somerst
pid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
527226040,195.0,41600,5,5,1969,1990,0.0,3,4,4,1100,3,1424,3,3,7,0,0,1990,1,3,828,144,11,2007,155000,0,1,0,1,0,0,2.5,0,1,1,1.0,5,False,False,False,True,False,False,False,False,False,False
916460020,68.0,7777,6,5,1996,1996,203.0,4,5,1,1491,5,1491,3,3,7,1,3,1996,3,2,571,0,11,2007,178000,1,0,1,1,1,1,2.0,1,1,1,1.0,1,False,False,False,False,False,False,False,False,False,False
907252060,68.0,12334,8,5,2003,2003,198.0,4,4,3,1068,5,2184,4,4,8,1,4,2003,2,2,570,192,6,2009,254750,1,0,1,1,1,1,2.5,1,1,1,2.0,1,False,True,False,False,False,False,False,False,False,False
534151120,68.0,13774,7,7,1977,1992,283.0,3,4,1,908,5,2288,4,4,8,2,3,1992,2,2,520,321,11,2009,230000,1,0,1,1,1,0,2.0,1,1,1,2.0,6,False,False,False,False,False,False,True,False,False,False
902406070,60.0,9720,6,7,1910,2002,0.0,3,3,1,741,5,1521,4,4,8,0,0,2002,1,3,640,0,9,2009,146500,0,0,1,0,1,0,1.0,1,2,0,2.0,1,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903451090,57.0,6876,6,5,1927,1950,0.0,3,3,1,720,2,1930,4,3,8,0,0,1927,1,1,316,0,8,2009,149000,0,1,0,1,1,0,3.0,1,2,0,2.0,3,False,False,False,False,False,False,False,False,False,False
528120120,90.0,11520,9,5,2005,2005,615.0,4,5,1,1698,5,1698,3,5,7,1,4,2005,3,3,730,192,6,2010,275000,1,0,1,1,1,1,2.0,1,1,0,1.0,6,False,False,False,False,False,False,False,False,False,False
528142150,90.0,12878,7,5,2003,2004,418.0,4,4,1,1760,5,1760,3,4,8,1,4,2004,3,2,583,165,6,2009,269500,1,0,1,1,0,1,3.0,1,1,1,1.0,5,False,False,False,False,False,False,False,False,False,False


In [ ]:
# generate a data frame of predictions using the ls_area_fit model and lad_area_fit model based on the ames_val_preprocessed data
pred_val_df = pd.DataFrame(
    {'true': ames_val_preprocessed['saleprice'],
     'ls_pred': ls_area_fit.predict(np.array(ames_val_preprocessed['gr_liv_area']).reshape(-1, 1)),
     'lad_pred': lad_area_fit.predict(np.array(ames_val_preprocessed['gr_liv_area']).reshape(-1, 1))})

In [ ]:
# look at the first few rows
pred_val_df.head()

,true,ls_pred,lad_pred
pid,,,
527226040,155000,169162.410072,167847.850055
916460020,178000,176490.766047,174348.401323
907252060,254750,252290.030090,241585.446527
534151120,230000,263665.388619,251675.854465
902406070,146500,179772.119469,177259.095921


Then we can compute the performance measures for our validation set predictions:

In [ ]:
# compute the rMSE, MAE, MAD, correlation and R2 of the true price with the LS and LAD predictions
print('LS rMSE:', np.sqrt(mean_squared_error(pred_val_df['true'], pred_val_df['ls_pred'])))
print('LS MAE:', mean_absolute_error(pred_val_df['true'], pred_val_df['ls_pred']))
print('LS MAD:', np.median(np.abs(pred_val_df['true'] - pred_val_df['ls_pred'])))
print('LS correlation:', np.corrcoef(pred_val_df['true'], pred_val_df['ls_pred'])[0, 1])
print('LS R2:', r2_score(pred_val_df['true'], pred_val_df['ls_pred']))

print('LAD rMSE:', np.sqrt(mean_squared_error(pred_val_df['true'], pred_val_df['lad_pred'])))
print('LAD MAE:', mean_absolute_error(pred_val_df['true'], pred_val_df['lad_pred']))
print('LAD MAD:', np.median(np.abs(pred_val_df['true'] - pred_val_df['lad_pred'])))
print('LAD correlation:', np.corrcoef(pred_val_df['true'], pred_val_df['lad_pred'])[0, 1])
print('LAD R2:', r2_score(pred_val_df['true'], pred_val_df['lad_pred']))



LS rMSE: 49376.502964829466
LS MAE: 34713.526131966435
LS MAD: 24150.199653045624
LS correlation: 0.6963124118184053
LS R2: 0.48365192100178145
LAD rMSE: 49506.79433630322
LAD MAE: 34303.04824461015
LAD MAD: 22192.392502773342
LAD correlation: 0.6963124118184056
LAD R2: 0.4809233170462617



Again, the performance for the two algorithms are very similar to one another, and again we see that the LS algorithm performs best in terms of the rMSE measure, but the LAD algorithm performs best in terms of the MAE and MAD measure (the two algorithms have identical performance in terms of the correlation and $R^2$ measures).

We also plot the predicted versus observed sale price responses for the training data, and add a diagonal line corresponding to the perfect prediction. Again, the two algorithms yield very similar predictions to one another on the validation set (just as they did on the training set). The performance on the validation set overall is a bit worse than the performance was on the training set houses (which is to be expected).


In [ ]:
# a scatterplots for each of the predicted and true sale price values for the LAD and LS algorithms applied to the validation data
lad_fig = px.scatter(pred_val_df, x='true', y='lad_pred')
# add a line with slope 1 and intercept 0
lad_fig.add_trace(
    go.Scatter(x=[0, 600000], y=[0, 600000], mode='lines'))
# update axes names and ranges
lad_fig.update_layout(xaxis_title='Observed sale price',
                        yaxis_title='Predicted sale price',
                        title="LAD pred",
                        xaxis_range=[0, 600000],
                        yaxis_range=[0, 600000])


In [ ]:
# a scatterplots for each of the predicted and true sale price values for the LAD and LS algorithms applied to the validation data
ls_fig = px.scatter(pred_val_df, x='true', y='ls_pred')
# add a line with slope 1 and intercept 0
ls_fig.add_trace(
    go.Scatter(x=[0, 600000], y=[0, 600000], mode='lines'))
# update axes names and ranges
ls_fig.update_layout(xaxis_title='Observed sale price',
                     yaxis_title='Predicted sale price',
                     title="LS pred",
                     xaxis_range=[0, 600000],
                     yaxis_range=[0, 600000])



#### Stability to data perturbations


To assess the stability of our data to appropriate perturbations in the data, we first need to decide what makes an "appropriate" perturbation. That is, what type of data perturbation (e.g., adding random noise, or performing sampling) most resembles the way that the data *could* have been measured or collected differently, as well as how these results will be applied in the future.


While the Ames housing data does not correspond to a random sample from a greater population of houses, each house is more-or-less exchangeable, meaning that a random sampling technique would be a reasonable perturbation, so we will draw 100 bootstrap samples of the original data.

Moreover, it is plausible that the living area measurements involve a slight amount of measurement error, although we do not have a realistic sense of how much. To really stress-test our results, we choose to add another perturbation to the data that involves adding some random noise to 30% of the `gr_liv_area` measurements. Since the standard deviation of the living area is approximately 500, we decide to add or subtract a random number between 0 and 250 (i.e. add noise up to half a standard deviation) to 30% of `gr_liv_area` observations.

Since we will be repeating this analysis many times, we will write a function that will take an Ames dataset, and return a perturbed version of it.


In [ ]:
# write a function that takes the ames_train_preprocessed data frame and creates a bootstrap sample of the same size
# and perturbs the gr_liv_area column by adding a random number between -250 and 250 to 30% of the values
def perturb_ames(df):
    # create a copy of the data frame
    df_copy = df.copy()
    # generate a random number between -250 and 250 for 30% of the rows
    sampled_index = df_copy.sample(frac=0.3).index
    df_copy.loc[sampled_index, 'gr_liv_area'] = df_copy.loc[sampled_index, 'gr_liv_area'] + np.random.randint(-250, 250, size=sampled_index.size)
    # conduct bootstrapping
    df_copy = df_copy.sample(frac=1, replace=True)
    return df_copy


Below we create a list containing the 100 perturbed versions of the training data.

In [ ]:
# create a list of 100 perturbed versions of ames_train_preprocessed using the perturb_ames function
perturbed_ames = [perturb_ames(ames_train_processed) for i in range(100)]

Then we can apply the LS and LAD fits to each of the 100 perturbed datasets.

In [ ]:
# apply LS and LAD to each of the 100 perturbed datasets
ls_perturbed = [LinearRegression().fit(X=np.array(df['gr_liv_area']).reshape(-1, 1), y=df['saleprice']) for df in perturbed_ames]
lad_perturbed = [LADRegression().fit(X=np.array(df['gr_liv_area']).reshape(-1, 1), y=df['saleprice']) for df in perturbed_ames]

We can then generate sale price predictions for each house in the validation set using each perturbed LS and LAD fit.

In [ ]:
# generate validation set sale price predictions for each of the 100 LS and LAD models
ls_perturbed_pred = [ls.predict(np.array(ames_val_preprocessed['gr_liv_area']).reshape(-1, 1)) for ls in ls_perturbed]
lad_perturbed_pred = [lad.predict(np.array(ames_val_preprocessed['gr_liv_area']).reshape(-1, 1)) for lad in lad_perturbed]

And compute the rMSE, MAE and $R^2$ for the validation set for each perturbed algorithm

In [ ]:
# compute the rMSE, MAE, MAD and correlation of the true price with the LS and LAD predictions for each of the 100 models
ls_perturbed_rmse = [np.sqrt(mean_squared_error(pred_val_df['true'], pred)) for pred in ls_perturbed_pred]
ls_perturbed_mae = [mean_absolute_error(pred_val_df['true'], pred) for pred in ls_perturbed_pred]
ls_perturbed_mad = [np.median(np.abs(pred_val_df['true'] - pred)) for pred in ls_perturbed_pred]
ls_perturbed_corr = [np.corrcoef(pred_val_df['true'], pred)[0, 1] for pred in ls_perturbed_pred]

lad_perturbed_rmse = [np.sqrt(mean_squared_error(pred_val_df['true'], pred)) for pred in lad_perturbed_pred]
lad_perturbed_mae = [mean_absolute_error(pred_val_df['true'], pred) for pred in lad_perturbed_pred]
lad_perturbed_mad = [np.median(np.abs(pred_val_df['true'] - pred)) for pred in lad_perturbed_pred]
lad_perturbed_corr = [np.corrcoef(pred_val_df['true'], pred)[0, 1] for pred in lad_perturbed_pred]


Then we can visualize the distribution of the performance metrics across the 100 perturbed iterations:

In [ ]:
# use boxplots to visualize the distribution of the performance measures for each model
# use plotly express and faceting to do this
ls_perturbed_df = pd.DataFrame({'rmse': ls_perturbed_rmse,
                                'mae': ls_perturbed_mae,
                                'mad': ls_perturbed_mad,
                                'corr': ls_perturbed_corr})
ls_perturbed_df['model'] = 'LS'

lad_perturbed_df = pd.DataFrame({'rmse': lad_perturbed_rmse,
                                    'mae': lad_perturbed_mae,
                                    'mad': lad_perturbed_mad,
                                    'corr': lad_perturbed_corr})
lad_perturbed_df['model'] = 'LAD'

perturbed_df = pd.concat([ls_perturbed_df, lad_perturbed_df])
perturbed_df

,rmse,mae,mad,corr,model
0,49379.859930,34367.644108,22504.703484,0.696312,LS
1,49327.305520,34600.154870,23698.440460,0.696312,LS
2,49414.671146,34847.519502,24162.956993,0.696312,LS
3,49389.305824,34772.240889,23870.131402,0.696312,LS
4,49347.950818,34509.428867,23808.418104,0.696312,LS
...,...,...,...,...,...
95,50222.937768,34357.840101,22686.842105,0.696312,LAD
96,49383.718970,34366.248023,22347.568988,0.696312,LAD
97,49687.952235,34288.939346,22339.015151,0.696312,LAD
98,49486.052178,34308.784540,22151.809356,0.696312,LAD


In [ ]:
px.box(perturbed_df, x='model', y='rmse', title="rMSE")

In [ ]:
px.box(perturbed_df, x='model', y='mae', title="MAE")

In [ ]:
px.box(perturbed_df, x='model', y='mad', title="MAD")

In [ ]:
px.box(perturbed_df, x='model', y='corr', title="Correlation", range_y=[0.65, 0.75])

Overall (by realizing that the range of each y-axis is fairly narrow), it seems like both algorithms are fairly stable in terms of these performance metrics.


Let's next examine the stability to data perturbations in terms of the individual predictions for individual validation set houses using prediction stability plots, which present the range of predictions for each individual observation on the x-axis as a line segment (ranging from the smallest to the largest). The intervals that contain the true response are colored blue.

To do this, we will write a function that will take the perturbed predictions data frame for each (validation) house, create a range of predictions, and produce a plot of the true response (y-axis) against the range of response predictions (x-axis).

In [ ]:
# define a function that takes a list of predictions from ls_perturbed_pred and creates line segment plots for the
# range of predictions for each id corresponding to the position in each list entry

def plot_prediction_range(pred_list, title=None):
    pred_list_df = pd.DataFrame(pred_list).T
    pred_list_df['id'] = ames_val_preprocessed.index
    pred_list_df['true'] = ames_val_preprocessed['saleprice'].values
    pred_list_df = pd.melt(pred_list_df, id_vars=['id','true'], var_name='iter', value_name='pred')
    pred_list_df = pred_list_df.groupby(['id', 'true']).agg({'pred': ['min', 'max']})
    pred_list_df = pred_list_df.reset_index()
    pred_list_df = pred_list_df.set_index('id')

    # plot a series of horizontal line segments for each id where the lines range from the minimum and maximum predicted values on the x-axis and have the true value on the y-axis
    fig = go.Figure()

    for i in pred_list_df.index:
        fig.add_trace(
            go.Scatter(x=[pred_list_df.loc[i, ('pred', 'min')], pred_list_df.loc[i, ('pred', 'max')]],
                        y=[pred_list_df.loc[i, 'true'].values[0], pred_list_df.loc[i, 'true'].values[0]],
                        mode='lines',
                        line={'color': 'black'})
            )
    # add a single diagonal line to the plot
    fig.add_trace(
        go.Scatter(x=[0, 550000], y=[0, 550000], mode='lines', line={'color': 'black'})
    )

    fig.update_layout(xaxis_title='Predicted sale price range',
                        yaxis_title='Observed sale price',
                        title=title)
    return fig


Then we can produce the prediction stability plots for LS and LAD as follows

In [ ]:
# use the function to plot the prediction range for the LS and LAD models
plot_prediction_range(ls_perturbed_pred, 'LS')

In [ ]:
plot_prediction_range(lad_perturbed_pred, 'LAD')


It seems like the range of predicted responses for each house is fairly narrow (i.e., the predictions are fairly stable), especially for houses in the middle of the price range, but is slightly wider (i.e., the predictions are somewhat less stable) for houses with higher predicted sale prices.